In [1]:
# Tế bào 1: Import các thư viện cần thiết
import chess
import chess.pgn
import json
from collections import defaultdict
import os # Để kiểm tra file tồn tại

print("Thư viện đã được import.")

Thư viện đã được import.


In [2]:
# Tế bào 2: Định nghĩa hàm tạo sách khai cuộc
def create_opening_book_from_pgn_notebook(pgn_file_path, max_ply=20, min_occurrence=5, book_file_path="opening_book_notebook.json"):
    """
    Xây dựng sách khai cuộc từ một file PGN trong Jupyter Notebook.

    Args:
        pgn_file_path (str): Đường dẫn đến file PGN.
        max_ply (int): Số ply tối đa (nửa nước đi) để xem xét cho khai cuộc.
        min_occurrence (int): Số lần xuất hiện tối thiểu của một nước đi để được đưa vào sách.
        book_file_path (str): Đường dẫn để lưu file sách khai cuộc (JSON).
    """
    # opening_book_data[fen_position_only][uci_move] = count
    opening_book_data = defaultdict(lambda: defaultdict(int))
    games_processed = 0
    corrupted_games_skipped = 0

    if not os.path.exists(pgn_file_path):
        print(f"LỖI: Không tìm thấy file PGN tại '{pgn_file_path}'")
        return None

    try:
        # Mở file PGN. 'errors="ignore"' có thể giúp bỏ qua các ký tự không hợp lệ
        # trong một số file PGN lớn, nhưng cũng có thể bỏ qua lỗi game.
        # Cân nhắc dùng 'replace' hoặc xử lý encoding cụ thể nếu biết.
        with open(pgn_file_path, 'r', encoding='utf-8', errors='ignore') as pgn_file:
            print(f"Bắt đầu xử lý file PGN: {pgn_file_path}")
            while True:
                game_headers = chess.pgn.read_headers(pgn_file)
                if game_headers is None:
                    break # Hết file

                # Cố gắng đọc phần còn lại của game
                # read_game sẽ đọc từ vị trí hiện tại của file handle
                # Nếu read_headers thành công, nó sẽ ở đầu các nước đi
                current_pos = pgn_file.tell()
                try:
                    # read_game sẽ đọc các nước đi sau headers
                    game = chess.pgn.read_game(pgn_file)
                    if game is None: # Trường hợp header cuối cùng không có game
                        # Cần đặt lại con trỏ file để read_headers lần sau không bị lỗi
                        pgn_file.seek(current_pos)
                        # Thử đọc lại game đầy đủ từ vị trí này, nếu không được thì bỏ qua
                        try:
                            game_to_check_end = chess.pgn.read_game(pgn_file)
                            if game_to_check_end is None: # Thực sự hết game
                                break
                            else: # Có game, đặt lại con trỏ
                                pgn_file.seek(current_pos)
                                game = chess.pgn.read_game(pgn_file) # Đọc lại
                        except Exception:
                            break # Coi như hết file

                except (ValueError, RuntimeError, AttributeError) as e: # Bắt các lỗi phổ biến khi đọc game
                    print(f"Lỗi khi đọc ván cờ thứ {games_processed + 1} (headers: {dict(game_headers)}): {e}. Bỏ qua ván này.")
                    corrupted_games_skipped += 1
                    # Cố gắng bỏ qua game lỗi và tìm game tiếp theo
                    # Điều này có thể khó nếu file PGN bị hỏng nặng
                    # read_headers ở lần lặp tiếp theo sẽ cố gắng tìm [Event
                    continue # Chuyển sang lần lặp while tiếp theo để đọc headers mới

                if game is None: # Nếu vẫn là None sau khi thử đọc lại
                    break

                games_processed += 1
                if games_processed % 100 == 0:
                    print(f"  Đã xử lý {games_processed} ván cờ...")

                board = game.board()
                ply_count = 0

                for move in game.mainline_moves():
                    if ply_count >= max_ply:
                        break

                    current_fen_full = board.fen()
                    # Sử dụng FEN chỉ vị trí quân làm khóa
                    # FEN: "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
                    # Lấy phần "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"
                    fen_parts = current_fen_full.split(' ')
                    current_fen_position_only = fen_parts[0]

                    # Lấy thêm thông tin lượt đi để làm khóa chính xác hơn
                    # key_fen = f"{fen_parts[0]} {fen_parts[1]}" # Vị trí + Lượt đi
                    key_fen = current_fen_position_only # Tạm thời vẫn dùng vị trí

                    uci_move = move.uci()

                    # Nếu bạn muốn sách khai cuộc chỉ cho một bên (ví dụ Trắng)
                    # if (board.turn == chess.WHITE and ply_count % 2 == 0) or \
                    #    (board.turn == chess.BLACK and ply_count % 2 != 0):
                    #     opening_book_data[key_fen][uci_move] += 1
                    # Hoặc lưu cho cả hai bên:
                    opening_book_data[key_fen][uci_move] += 1


                    try:
                        board.push(move)
                    except Exception as e_push:
                        print(f"Lỗi khi thực hiện nước đi {uci_move} trong ván {games_processed}: {e_push}")
                        break

                    ply_count += 1

    except Exception as e_open:
        print(f"Lỗi nghiêm trọng khi mở hoặc xử lý file PGN: {e_open}")
        return None

    print(f"\nHoàn thành phân tích. Tổng số ván đã xử lý (thành công): {games_processed}")
    if corrupted_games_skipped > 0:
        print(f"Số ván cờ bị lỗi và bỏ qua: {corrupted_games_skipped}")
    print(f"Số lượng thế cờ gốc trong sách thô: {len(opening_book_data)}")

    filtered_opening_book = {}
    for fen, moves_with_counts in opening_book_data.items():
        valid_moves_for_fen = {
            move_uci: count for move_uci, count in moves_with_counts.items() if count >= min_occurrence
        }
        if valid_moves_for_fen:
            # Lưu dưới dạng list các (move, count) để có thể chọn theo trọng số sau này
            # Hoặc chỉ list các move nếu chỉ chọn ngẫu nhiên
            # filtered_opening_book[fen] = list(valid_moves_for_fen.keys()) # Chỉ nước đi

            # Lưu cả nước đi và tần suất để có thể chọn lựa thông minh hơn
            sorted_moves = sorted(valid_moves_for_fen.items(), key=lambda item: item[1], reverse=True)
            filtered_opening_book[fen] = sorted_moves # List of [("e2e4", 50), ("d2d4", 30)]

    print(f"Số lượng thế cờ trong sách đã lọc (min_occurrence={min_occurrence}): {len(filtered_opening_book)}")

    try:
        with open(book_file_path, 'w') as f:
            json.dump(filtered_opening_book, f, indent=2) # indent=2 cho dễ đọc hơn
        print(f"Sách khai cuộc đã được lưu vào: {book_file_path}")
    except Exception as e_save:
        print(f"Lỗi khi lưu sách khai cuộc: {e_save}")

    return filtered_opening_book

print("Hàm tạo sách khai cuộc đã được định nghĩa.")

Hàm tạo sách khai cuộc đã được định nghĩa.


In [3]:
# Tế bào 3: Thiết lập đường dẫn và tham số, sau đó chạy hàm

# --- THAY ĐỔI CÁC THAM SỐ NÀY CHO PHÙ HỢP ---
# Đường dẫn đến file PGN của bạn.
# Nếu file lớn, quá trình này có thể mất nhiều thời gian.
# Hãy thử với một file PGN nhỏ trước (vài trăm hoặc vài nghìn ván).
PGN_DATABASE_FILE = "D:/AI/Chess/ChessApp/Chess/raw_pgn/Modern.pgn"  # Ví dụ, bạn cần thay đổi
# PGN_DATABASE_FILE = "small_test.pgn" # Tạo một file pgn nhỏ để test nhanh

OUTPUT_BOOK_FILE = "D:/AI/Chess/ChessApp/Chess..pgnopening_book_from_pgn.json"

# Số ply (nửa nước đi) tối đa để đưa vào sách.
# 20 ply = 10 nước đi của Trắng và 10 nước đi của Đen.
MAX_PLY_FOR_BOOK_GENERATION = 16

# Một nước đi phải xuất hiện ít nhất bao nhiêu lần từ một thế cờ để được coi là "đáng tin cậy".
MIN_MOVE_OCCURRENCE = 5
# Nếu PGN của bạn rất lớn, bạn có thể tăng giá trị này.
# Nếu PGN nhỏ, bạn có thể giảm xuống.

# --- Chạy hàm tạo sách ---
print(f"Bắt đầu tạo sách khai cuộc từ: {PGN_DATABASE_FILE}")
print(f"Max ply: {MAX_PLY_FOR_BOOK_GENERATION}, Min occurrence: {MIN_MOVE_OCCURRENCE}")

opening_book_generated = create_opening_book_from_pgn_notebook(
    PGN_DATABASE_FILE,
    max_ply=MAX_PLY_FOR_BOOK_GENERATION,
    min_occurrence=MIN_MOVE_OCCURRENCE,
    book_file_path=OUTPUT_BOOK_FILE
)

if opening_book_generated:
    print("\nTạo sách khai cuộc hoàn thành.")
    # In ra một vài mục đầu tiên của sách để kiểm tra
    print("\nVí dụ một vài mục trong sách:")
    count = 0
    for fen_pos, moves_list in opening_book_generated.items():
        print(f"FEN (vị trí): {fen_pos}")
        for move_uci, move_count in moves_list:
            print(f"  -> {move_uci} (xuất hiện {move_count} lần)")
        count += 1
        if count >= 3: # Chỉ in 3 mục đầu
            break
else:
    print("\nKhông thể tạo sách khai cuộc.")

Bắt đầu tạo sách khai cuộc từ: D:/AI/Chess/ChessApp/Chess/raw_pgn/Modern.pgn
Max ply: 16, Min occurrence: 5
Bắt đầu xử lý file PGN: D:/AI/Chess/ChessApp/Chess/raw_pgn/Modern.pgn
  Đã xử lý 100 ván cờ...
  Đã xử lý 200 ván cờ...
  Đã xử lý 300 ván cờ...
  Đã xử lý 400 ván cờ...
  Đã xử lý 500 ván cờ...
  Đã xử lý 600 ván cờ...
  Đã xử lý 700 ván cờ...
  Đã xử lý 800 ván cờ...
  Đã xử lý 900 ván cờ...
  Đã xử lý 1000 ván cờ...
  Đã xử lý 1100 ván cờ...
  Đã xử lý 1200 ván cờ...
  Đã xử lý 1300 ván cờ...
  Đã xử lý 1400 ván cờ...
  Đã xử lý 1500 ván cờ...
  Đã xử lý 1600 ván cờ...
  Đã xử lý 1700 ván cờ...
  Đã xử lý 1800 ván cờ...
  Đã xử lý 1900 ván cờ...
  Đã xử lý 2000 ván cờ...
  Đã xử lý 2100 ván cờ...
  Đã xử lý 2200 ván cờ...
  Đã xử lý 2300 ván cờ...
  Đã xử lý 2400 ván cờ...
  Đã xử lý 2500 ván cờ...
  Đã xử lý 2600 ván cờ...
  Đã xử lý 2700 ván cờ...
  Đã xử lý 2800 ván cờ...
  Đã xử lý 2900 ván cờ...
  Đã xử lý 3000 ván cờ...
  Đã xử lý 3100 ván cờ...
  Đã xử lý 3200 ván cờ...